<font color='dimgrey'> Recieves all the fibonacci call requests here '\#' </font> 

In [3]:
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()
# server declares the rpc_queue. Needs to start first
channel.queue_declare(queue='rpc_queue')

#finds fibonacci
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

# when recieving a request from the client
def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)
    
    # response is published to the callback queue stored in the reply_to arg
    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
try:
    channel.start_consuming()
except KeyboardInterrupt:
    # Exit gracefully
    connection.close()

 [x] Awaiting RPC requests
 [.] fib(35)
 [.] fib(40)
 [.] fib(300)
